# Angular Velocity: Intuitive 3D Animations with Real Meshes
This notebook upgrades the earlier matplotlib-only plots to **mesh-based, interactive 3D** (Plotly) visuals.

You’ll see angular velocity as:
- a **rotation axis + rate** (vector \(\omega\))
- the generator of tangential motion: \(\dot p = \omega \times p\)
- the generator of orientation: \(\dot R = [\omega]R\)

## What you’ll get
1) **Basic 3D**: a rotating arrow + tangent velocity arrow
2) **Airplane mesh**: yaw/pitch/roll + a point on the wing showing \(v=\omega\times r\)
3) **Robotics mesh**: a gripper/hand mesh spinning with a chosen \(\omega\), with axes + tangent velocity

### Mesh sources
- Aircraft models: Flightradar24 `fr24-3d-models` (glTF) citeturn0search1
- Robot meshes: MoveIt `moveit_resources` Panda (meshes) citeturn0search6turn0search2

> If you're offline or `git` is blocked, you can skip the mesh cells and still run the Basic 3D section.

In [ ]:
# --- Dependencies ---
# If you don't have git available, you can manually download the repos instead.
import os, glob, math, subprocess, shutil
import numpy as np

import plotly.graph_objects as go
import trimesh

np.set_printoptions(precision=4, suppress=True)

## Helpers (math + Plotly mesh/axes)

In [ ]:
def unit(v, eps=1e-12):
    v = np.asarray(v, dtype=float).reshape(-1)
    n = np.linalg.norm(v)
    return v if n < eps else v/n

def skew(w):
    w1,w2,w3 = np.asarray(w, dtype=float).reshape(3,)
    return np.array([[0, -w3, w2],
                     [w3, 0, -w1],
                     [-w2, w1, 0]], dtype=float)

def rodrigues(axis, theta):
    axis = unit(axis)
    K = skew(axis)
    I = np.eye(3)
    return I + math.sin(theta)*K + (1-math.cos(theta))*(K@K)

def apply_R(R, V):
    return (R @ np.asarray(V).T).T

def mesh_to_plotly(mesh: trimesh.Trimesh, name="mesh", opacity=1.0):
    # Plotly Mesh3d wants vertices + triangular faces
    v = mesh.vertices
    f = mesh.faces
    return go.Mesh3d(
        x=v[:,0], y=v[:,1], z=v[:,2],
        i=f[:,0], j=f[:,1], k=f[:,2],
        name=name,
        opacity=opacity,
        flatshading=True,
    )

def axes_traces(R=np.eye(3), origin=(0,0,0), L=1.0, name_prefix=""):
    o = np.array(origin, float)
    ex, ey, ez = R[:,0], R[:,1], R[:,2]
    traces = []
    for vec, nm in [(ex, "x"), (ey, "y"), (ez, "z")]:
        p = o + L*vec
        traces.append(go.Scatter3d(
            x=[o[0], p[0]], y=[o[1], p[1]], z=[o[2], p[2]],
            mode="lines",
            name=f"{name_prefix}{nm}",
            line=dict(width=6),
            showlegend=False
        ))
    return traces

def arrow_trace(vec, origin=(0,0,0), name="arrow", scale=1.0, width=8):
    o = np.array(origin, float)
    v = np.array(vec, float)*scale
    p = o + v
    return go.Scatter3d(
        x=[o[0], p[0]], y=[o[1], p[1]], z=[o[2], p[2]],
        mode="lines+markers",
        name=name,
        marker=dict(size=3),
        line=dict(width=width),
        showlegend=False
    )

def set_scene(fig, lim=2.0, title=""):
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis=dict(range=[-lim, lim]),
            yaxis=dict(range=[-lim, lim]),
            zaxis=dict(range=[-lim, lim]),
            aspectmode="cube"
        ),
        margin=dict(l=0, r=0, t=40, b=0)
    )
    return fig

## Fetch meshes (airplane + robot hand)
We’ll `git clone` two repos and then **auto-pick a reasonable mesh file**.

- Aircraft repo: Flightradar24 glTF models citeturn0search1
- Robot repo: MoveIt resources (Franka Panda meshes) citeturn0search6turn0search2


In [ ]:
ASSETS_DIR = "assets_meshes"
os.makedirs(ASSETS_DIR, exist_ok=True)

def git_clone_if_needed(url, dst, depth=1):
    if os.path.exists(dst) and os.path.isdir(dst) and os.listdir(dst):
        return
    if os.path.exists(dst):
        shutil.rmtree(dst)
    subprocess.check_call(["git", "clone", "--depth", str(depth), url, dst])

air_repo = os.path.join(ASSETS_DIR, "fr24-3d-models")
robot_repo = os.path.join(ASSETS_DIR, "moveit_resources")

# You can comment out any clone if it fails in your environment
git_clone_if_needed("https://github.com/Flightradar24/fr24-3d-models.git", air_repo)
git_clone_if_needed("https://github.com/moveit/moveit_resources.git", robot_repo)

# --- Find an aircraft glTF/glb (first match) ---
air_candidates = glob.glob(os.path.join(air_repo, "**", "*.glb"), recursive=True) + \
                 glob.glob(os.path.join(air_repo, "**", "*.gltf"), recursive=True)

# Prefer smaller-ish filenames if possible (heuristic)
air_candidates = sorted(air_candidates, key=lambda p: (os.path.getsize(p), len(p)))

# #region agent log
import json; f=open(r'\\wsl$\Ubuntu\home\villy\code\modern-robotics-labbook\.cursor\debug.log','a'); f.write(json.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"A","location":"cell5:file_selection","message":"File candidates found","data":{"count":len(air_candidates),"first_candidate":air_candidates[0] if air_candidates else None},"timestamp":int(__import__('time').time()*1000)})+'\n'); f.close()
# #endregion

air_path = air_candidates[0] if air_candidates else None
print("Aircraft model:", air_path)

# --- Find a Panda hand / gripper mesh (STL/DAE) ---
robot_candidates = glob.glob(os.path.join(robot_repo, "**", "*hand*.stl"), recursive=True) + \
                   glob.glob(os.path.join(robot_repo, "**", "*gripper*.stl"), recursive=True) + \
                   glob.glob(os.path.join(robot_repo, "**", "*hand*.dae"), recursive=True)

robot_candidates = sorted(robot_candidates, key=lambda p: (os.path.getsize(p), len(p)))
robot_path = robot_candidates[0] if robot_candidates else None
print("Robot hand/gripper mesh:", robot_path)

In [ ]:
import os, glob, json, struct

def is_gltf2(path: str) -> bool:
    # #region agent log
    import json as json_mod, time; f=open(r'\\wsl$\Ubuntu\home\villy\code\modern-robotics-labbook\.cursor\debug.log','a'); f.write(json_mod.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"A","location":"is_gltf2:entry","message":"Checking GLTF version","data":{"path":path},"timestamp":int(time.time()*1000)})+'\n'); f.close()
    # #endregion
    ext = os.path.splitext(path)[1].lower()
    # #region agent log
    f=open(r'\\wsl$\Ubuntu\home\villy\code\modern-robotics-labbook\.cursor\debug.log','a'); f.write(json_mod.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"A","location":"is_gltf2:extension","message":"File extension","data":{"ext":ext},"timestamp":int(time.time()*1000)})+'\n'); f.close()
    # #endregion

    if ext == ".gltf":
        try:
            # #region agent log
            f=open(r'\\wsl$\Ubuntu\home\villy\code\modern-robotics-labbook\.cursor\debug.log','a'); f.write(json_mod.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"B","location":"is_gltf2:gltf_try","message":"Trying to read GLTF JSON","data":{"path":path},"timestamp":int(time.time()*1000)})+'\n'); f.close()
            # #endregion
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
            v = str(data.get("asset", {}).get("version", ""))
            # #region agent log
            f=open(r'\\wsl$\Ubuntu\home\villy\code\modern-robotics-labbook\.cursor\debug.log','a'); f.write(json_mod.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"B","location":"is_gltf2:gltf_version","message":"GLTF version found","data":{"version":v,"starts_with_2":v.startswith("2")},"timestamp":int(time.time()*1000)})+'\n'); f.close()
            # #endregion
            return v.startswith("2")
        except Exception as e:
            # #region agent log
            f=open(r'\\wsl$\Ubuntu\home\villy\code\modern-robotics-labbook\.cursor\debug.log','a'); f.write(json_mod.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"C","location":"is_gltf2:gltf_exception","message":"Exception reading GLTF","data":{"path":path,"error":str(e),"error_type":type(e).__name__},"timestamp":int(time.time()*1000)})+'\n'); f.close()
            # #endregion
            return False

    if ext == ".glb":
        try:
            # #region agent log
            f=open(r'\\wsl$\Ubuntu\home\villy\code\modern-robotics-labbook\.cursor\debug.log','a'); f.write(json_mod.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"D","location":"is_gltf2:glb_try","message":"Trying to read GLB binary","data":{"path":path},"timestamp":int(time.time()*1000)})+'\n'); f.close()
            # #endregion
            with open(path, "rb") as f:
                header = f.read(4)
                # #region agent log
                log_f=open(r'\\wsl$\Ubuntu\home\villy\code\modern-robotics-labbook\.cursor\debug.log','a'); log_f.write(json_mod.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"D","location":"is_gltf2:glb_header","message":"GLB header read","data":{"header":header.hex(),"is_gltf":header==b"glTF"},"timestamp":int(time.time()*1000)})+'\n'); log_f.close()
                # #endregion
                if header != b"glTF":
                    return False
                version_bytes = f.read(4)
                version = struct.unpack("<I", version_bytes)[0]
                # #region agent log
                log_f=open(r'\\wsl$\Ubuntu\home\villy\code\modern-robotics-labbook\.cursor\debug.log','a'); log_f.write(json_mod.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"D","location":"is_gltf2:glb_version","message":"GLB version read","data":{"version":version,"version_bytes_hex":version_bytes.hex(),"is_v2":version==2},"timestamp":int(time.time()*1000)})+'\n'); log_f.close()
                # #endregion
            return version == 2
        except Exception as e:
            # #region agent log
            f=open(r'\\wsl$\Ubuntu\home\villy\code\modern-robotics-labbook\.cursor\debug.log','a'); f.write(json_mod.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"E","location":"is_gltf2:glb_exception","message":"Exception reading GLB","data":{"path":path,"error":str(e),"error_type":type(e).__name__},"timestamp":int(time.time()*1000)})+'\n'); f.close()
            # #endregion
            return False

    # #region agent log
    f=open(r'\\wsl$\Ubuntu\home\villy\code\modern-robotics-labbook\.cursor\debug.log','a'); f.write(json_mod.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"A","location":"is_gltf2:not_gltf","message":"Not a GLTF/GLB file","data":{"ext":ext},"timestamp":int(time.time()*1000)})+'\n'); f.close()
    # #endregion
    return False

# Find glTF2 aircraft meshes
air_candidates = (
    glob.glob(os.path.join(air_repo, "**", "*.glb"), recursive=True) +
    glob.glob(os.path.join(air_repo, "**", "*.gltf"), recursive=True)
)
# #region agent log
import json as json_mod, time; f=open(r'\\wsl$\Ubuntu\home\villy\code\modern-robotics-labbook\.cursor\debug.log','a'); f.write(json_mod.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"F","location":"cell6:before_filter","message":"Files found before filtering","data":{"count":len(air_candidates),"first_few":air_candidates[:5]},"timestamp":int(time.time()*1000)})+'\n'); f.close()
# #endregion
filtered_candidates = []
for p in air_candidates:
    result = is_gltf2(p)
    # #region agent log
    f=open(r'\\wsl$\Ubuntu\home\villy\code\modern-robotics-labbook\.cursor\debug.log','a'); f.write(json_mod.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"F","location":"cell6:filter_result","message":"Filter result","data":{"path":p,"is_gltf2":result},"timestamp":int(time.time()*1000)})+'\n'); f.close()
    # #endregion
    if result:
        filtered_candidates.append(p)
air_candidates = filtered_candidates
air_candidates = sorted(air_candidates, key=lambda p: (os.path.getsize(p), len(p)))
# #region agent log
f=open(r'\\wsl$\Ubuntu\home\villy\code\modern-robotics-labbook\.cursor\debug.log','a'); f.write(json_mod.dumps({"sessionId":"debug-session","runId":"run1","hypothesisId":"F","location":"cell6:after_filter","message":"Files after filtering","data":{"count":len(air_candidates)},"timestamp":int(time.time()*1000)})+'\n'); f.close()
# #endregion

air_path = air_candidates[0] if air_candidates else None
print("Picked aircraft model (glTF2):", air_path)

if air_path is None:
    raise RuntimeError("No glTF2 aircraft model found. Need a glTF 2.0 .glb/.gltf for trimesh.")


## Load meshes (and lightly normalize scale)
We’ll scale each mesh to roughly fit in a \([-1,1]\) cube for nice viewing.

In [ ]:
def load_mesh_any(path):
    if path is None:
        return None
    mesh_or_scene = trimesh.load(path)
    if isinstance(mesh_or_scene, trimesh.Scene):
        # combine all geometries into one mesh for simplicity
        mesh = trimesh.util.concatenate([g for g in mesh_or_scene.geometry.values()])
    else:
        mesh = mesh_or_scene
    mesh = mesh.copy()
    # These methods don't exist in all trimesh versions, so check before calling
    if hasattr(mesh, 'remove_unreferenced_vertices'):
        mesh.remove_unreferenced_vertices()
    if hasattr(mesh, 'remove_duplicate_faces'):
        mesh.remove_duplicate_faces()
    if hasattr(mesh, 'remove_degenerate_faces'):
        mesh.remove_degenerate_faces()
    return mesh

def normalize_mesh(mesh: trimesh.Trimesh, target=1.0):
    if mesh is None: 
        return None
    mesh = mesh.copy()
    # center
    c = mesh.bounding_box.centroid
    mesh.apply_translation(-c)
    # scale so that max extent ~ target*2
    ext = mesh.extents
    s = (2*target) / (np.max(ext) + 1e-12)
    mesh.apply_scale(s)
    return mesh

air_mesh = normalize_mesh(load_mesh_any(air_path), target=1.0)
robot_mesh = normalize_mesh(load_mesh_any(robot_path), target=0.9)

print("air mesh:", None if air_mesh is None else (len(air_mesh.vertices), len(air_mesh.faces)))
print("robot mesh:", None if robot_mesh is None else (len(robot_mesh.vertices), len(robot_mesh.faces)))

## Demo 1 — Basic 3D: angular velocity makes tangents
We rotate a vector \(p(t)\) and show the instantaneous tangent velocity:
\[\dot p = \omega \times p\]

Use the slider to scrub time.

In [ ]:
# Parameters
omega = np.array([0.2, 0.6, 1.0])   # rad/s (space coordinates)
p0 = unit([1.0, 0.3, 0.0])

T = 4.0
N = 80
ts = np.linspace(0, T, N)

frames = []
for t in ts:
    R = rodrigues(omega, np.linalg.norm(omega)*t)  # rotate about omega direction
    p_t = R @ p0
    v_t = np.cross(omega, p_t)
    v_vis = 0.5 * v_t/(np.linalg.norm(v_t)+1e-12)

    data = []
    data += axes_traces(np.eye(3), L=1.2, name_prefix="space_")
    data.append(arrow_trace(omega, name="ω", scale=0.5, width=10))
    data.append(arrow_trace(p_t, name="p(t)", scale=1.0, width=12))
    data.append(arrow_trace(v_vis, origin=p_t, name="tangent v", scale=1.0, width=8))

    frames.append(go.Frame(data=data, name=f"{t:.2f}"))

fig = go.Figure(frames=frames)
fig.add_traces(frames[0].data)
set_scene(fig, lim=1.6, title="Demo 1: p(t) and tangent v = ω × p")

fig.update_layout(
    updatemenus=[dict(
        type="buttons",
        buttons=[
            dict(label="Play", method="animate", args=[None, {"frame": {"duration": 40, "redraw": True}, "fromcurrent": True}]),
            dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]),
        ],
        x=0.02, y=0.02
    )],
    sliders=[dict(
        steps=[dict(method="animate", args=[[f.name], {"mode":"immediate", "frame":{"duration":0,"redraw":True}}], label=f.name) for f in frames],
        x=0.1, y=0.02, len=0.85
    )]
)
fig

## Demo 2 — Airplane mesh: yaw/pitch/roll + point velocity
We attach an airplane mesh to a body frame. Then we apply yaw/pitch/roll over time.

Key idea: pick a point on the body (e.g., wing tip). Its instantaneous velocity from rotation is:
\[\;v = \omega \times r\;\]
where \(r\) is the point position relative to the rotation origin.

Use the slider to scrub time.

In [ ]:
if air_mesh is None:
    raise RuntimeError("No airplane mesh found. Check that the aircraft repo cloned successfully.")

# Choose a point on the mesh as 'wing tip' (heuristic: farthest +y)
V = air_mesh.vertices
wing_tip = V[np.argmax(V[:,1])]

# Time-varying yaw/pitch/roll (radians)
T = 6.0
N = 90
ts = np.linspace(0, T, N)

# Define yaw/pitch/roll trajectories
yaw = np.deg2rad(40)*np.sin(2*np.pi*ts/T)
pitch = np.deg2rad(20)*np.sin(2*np.pi*ts/T + 0.6)
roll = np.deg2rad(30)*np.sin(2*np.pi*ts/T + 1.2)

# Approximate ω from finite differences of R(t):  [ω] ≈ Rdot R^T
# (space form)
frames = []

# Precompute rotation matrices
Rs = []
for i,t in enumerate(ts):
    R = (rodrigues([0,0,1], yaw[i]) @
         rodrigues([0,1,0], pitch[i]) @
         rodrigues([1,0,0], roll[i]))
    Rs.append(R)
Rs = np.array(Rs)

dt = ts[1]-ts[0]
for i,t in enumerate(ts):
    R = Rs[i]
    # mesh transformed
    Vt = apply_R(R, air_mesh.vertices)
    mesh_t = trimesh.Trimesh(vertices=Vt, faces=air_mesh.faces, process=False)

    # estimate ω (space) using Rdot R^T
    if 0 < i < len(ts)-1:
        Rdot = (Rs[i+1]-Rs[i-1])/(2*dt)
    elif i == 0:
        Rdot = (Rs[1]-Rs[0])/dt
    else:
        Rdot = (Rs[-1]-Rs[-2])/dt
    omega_hat = Rdot @ R.T
    omega_s = np.array([omega_hat[2,1], omega_hat[0,2], omega_hat[1,0]])

    # wing tip position and velocity arrow
    pt = R @ wing_tip
    vt = np.cross(omega_s, pt)
    vt_vis = 0.4 * vt/(np.linalg.norm(vt)+1e-12)

    data = []
    data.append(mesh_to_plotly(mesh_t, name="airplane", opacity=0.85))
    data += axes_traces(np.eye(3), L=1.2, name_prefix="space_")
    data += axes_traces(R, L=1.0, name_prefix="body_")
    data.append(arrow_trace(omega_s, name="ω (estimated)", scale=0.4, width=10))
    data.append(arrow_trace(vt_vis, origin=pt, name="v at wing tip", scale=1.0, width=8))
    data.append(go.Scatter3d(x=[pt[0]], y=[pt[1]], z=[pt[2]], mode="markers", marker=dict(size=5), showlegend=False))

    frames.append(go.Frame(data=data, name=f"{t:.2f}"))

fig = go.Figure(frames=frames)
fig.add_traces(frames[0].data)
set_scene(fig, lim=1.8, title="Demo 2: Airplane yaw/pitch/roll and v = ω × r (wing tip)")

fig.update_layout(
    updatemenus=[dict(
        type="buttons",
        buttons=[
            dict(label="Play", method="animate", args=[None, {"frame": {"duration": 45, "redraw": True}, "fromcurrent": True}]),
            dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]),
        ],
        x=0.02, y=0.02
    )],
    sliders=[dict(
        steps=[dict(method="animate", args=[[f.name], {"mode":"immediate", "frame":{"duration":0,"redraw":True}}], label=f.name) for f in frames],
        x=0.1, y=0.02, len=0.85
    )]
)
fig

## Demo 3 — Robotics mesh: gripper/hand mesh + ω + tangent velocity
We load a gripper/hand mesh (from MoveIt Panda resources) and spin it about an axis.
We again show a body point’s instantaneous velocity: \(v = \omega 	imes r\).

Use the slider to scrub time.

In [ ]:
if robot_mesh is None:
    raise RuntimeError("No robot hand/gripper mesh found. Check that moveit_resources cloned successfully.")

# pick a point on the mesh as 'fingertip' (heuristic: farthest +x)
V = robot_mesh.vertices
tip = V[np.argmax(V[:,0])]

omega = np.array([0.0, 1.2, 0.6])  # rad/s
T = 4.0
N = 80
ts = np.linspace(0, T, N)

frames = []
for t in ts:
    theta = np.linalg.norm(omega)*t
    R = rodrigues(omega, theta)

    Vt = apply_R(R, robot_mesh.vertices)
    mesh_t = trimesh.Trimesh(vertices=Vt, faces=robot_mesh.faces, process=False)

    pt = R @ tip
    vt = np.cross(omega, pt)
    vt_vis = 0.35 * vt/(np.linalg.norm(vt)+1e-12)

    data = []
    data.append(mesh_to_plotly(mesh_t, name="robot hand", opacity=0.9))
    data += axes_traces(np.eye(3), L=1.2, name_prefix="space_")
    data += axes_traces(R, L=1.0, name_prefix="body_")
    data.append(arrow_trace(omega, name="ω", scale=0.4, width=10))
    data.append(go.Scatter3d(x=[pt[0]], y=[pt[1]], z=[pt[2]], mode="markers", marker=dict(size=5), showlegend=False))
    data.append(arrow_trace(vt_vis, origin=pt, name="v at tip", scale=1.0, width=8))

    frames.append(go.Frame(data=data, name=f"{t:.2f}"))

fig = go.Figure(frames=frames)
fig.add_traces(frames[0].data)
set_scene(fig, lim=1.6, title="Demo 3: Robot hand mesh spinning; v = ω × r (tip)")

fig.update_layout(
    updatemenus=[dict(
        type="buttons",
        buttons=[
            dict(label="Play", method="animate", args=[None, {"frame": {"duration": 45, "redraw": True}, "fromcurrent": True}]),
            dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]),
        ],
        x=0.02, y=0.02
    )],
    sliders=[dict(
        steps=[dict(method="animate", args=[[f.name], {"mode":"immediate", "frame":{"duration":0,"redraw":True}}], label=f.name) for f in frames],
        x=0.1, y=0.02, len=0.85
    )]
)
fig

## Notes on “real models people use” (quick)
- **Robotics**: URDF + meshes (STL/DAE/OBJ). Popular sources include MoveIt resources, ROS-Industrial packages, manufacturer-provided CAD exports.
- **Aircraft**: glTF/OBJ/FBX meshes (FlightGear / Flightradar24 / etc.).

If you want, we can extend this notebook to load a **full URDF** (entire robot arm) and animate joint motions + spatial/angular velocity of each link.
